In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import scipy
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
import xgboost as xgb

In [2]:
train_df = pd.read_csv("data/train_modified.csv", index_col="id")
test_df = pd.read_csv("data/test.csv", index_col="id")
oil_df = pd.read_csv("data/oil.csv")

In [3]:
train_df["date"] = pd.to_datetime(train_df["date"])
train_df["store_nbr"] = train_df["store_nbr"].astype("int16")
train_df["family"] = train_df["family"].astype("category")
train_df["sales"] = train_df["sales"].astype("float32")
train_df["onpromotion"] = train_df["onpromotion"].astype("float16")
train_df["month"] = train_df["month"].astype("category")
train_df["day"] = train_df["day"].astype("category")
train_df["week_part"] = train_df["week_part"].astype("category")
train_df["days_since_last_promo"] = train_df["days_since_last_promo"].astype("int16")
train_df["bin_last_promo"] = train_df["bin_last_promo"].astype("category")
train_df["locale"] = train_df["locale"].astype("category")
train_df["type"] = train_df["type"].astype("category")
train_df["cluster"] = train_df["cluster"].astype("category")
oil_df["date"] = pd.to_datetime(oil_df["date"])

In [4]:
train_df = pd.merge(train_df, oil_df, on="date", how="left")
train_df["dcoilwtico"] = train_df["dcoilwtico"].fillna(method="ffill")
train_df["dcoilwtico"] = train_df["dcoilwtico"].fillna(method="bfill")

In [5]:
train_df["weekend"] = train_df['week_part'].apply(lambda x: x == 'Weekend')
train_df["weekend"] = train_df["weekend"].astype("bool")
train_df = train_df.drop("week_part", axis=1)

In [6]:
test_df["date"] = pd.to_datetime(test_df["date"])
test_df = pd.merge(test_df, oil_df, on="date", how="left")
test_df["dcoilwtico"] = test_df["dcoilwtico"].fillna(method="ffill")
test_df["year"] = test_df["date"].dt.year
test_df["month"] = test_df["date"].dt.month
test_df["day"] = test_df["date"].dt.day

### Panel Regression

In [7]:
num_features = ["store_nbr", "onpromotion", "is_holiday", "weekend", "dcoilwtico"]
cat_features = ["family"]

In [8]:
X = train_df.drop("sales", axis=1)[["date"] + num_features + cat_features]
X.set_index(["store_nbr", "date"], inplace=True)
X = pd.get_dummies(X)
y = train_df[["sales", "store_nbr", "date"]]
y.set_index(["store_nbr", "date"], inplace=True)

In [9]:
panel_regression_model = PanelOLS(y, X, entity_effects=True, time_effects=True, check_rank=False, drop_absorbed=True)
panel_model = panel_regression_model.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)

/var/folders/14/8l60xyhn23vgqk95vx1p0rv00000gn/T/ipykernel_1763/3671716052.py:2: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

is_holiday, weekend, family_SEAFOOD

  panel_model = panel_regression_model.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)


In [10]:
print(panel_model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  sales   R-squared:                        0.5596
Estimator:                   PanelOLS   R-squared (Between):              0.6112
No. Observations:             3054348   R-squared (Within):               0.5479
Date:                Wed, Feb 22 2023   R-squared (Overall):              0.5566
Time:                        12:56:53   Log-likelihood                 -2.44e+07
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   1.141e+05
Entities:                          54   P-value                           0.0000
Avg Obs:                    5.656e+04   Distribution:              F(34,3052577)
Min Obs:                    5.656e+04                                           
Max Obs:                    5.656e+04   F-statistic (robust):             40.022
                            

/Users/marklenkei/opt/miniconda3/envs/sales-forecasting/lib/python3.10/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/Users/marklenkei/opt/miniconda3/envs/sales-forecasting/lib/python3.10/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/Users/marklenkei/opt/miniconda3/envs/sales-forecasting/lib/python3.10/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/Users/marklenkei/opt/miniconda3/envs/sales-forecasting/lib/python3.10/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


In [11]:
panel_test = test_df[["date", "store_nbr", "onpromotion", "dcoilwtico", "family"]]
panel_test.set_index(["store_nbr", "date"], inplace=True)
panel_test = pd.get_dummies(panel_test)
panel_test = panel_test.drop("family_SEAFOOD", axis=1)

In [12]:
panel_pred = panel_model.predict(panel_test)

In [13]:
len(panel_pred)

28512

In [14]:
panel_pred

predictions
store_nbr date                   
1         2017-08-16    95.915999
          2017-08-16    90.855858
          2017-08-16   127.002007
          2017-08-16  2660.507984
          2017-08-16    90.825358
...                           ...
9         2017-08-31   416.590572
          2017-08-31   181.460038
          2017-08-31  1251.323797
          2017-08-31   250.142745
          2017-08-31   106.359217

[28512 rows x 1 columns]

### XGBoost

In [15]:
num_features = ["store_nbr", "onpromotion", "dcoilwtico"]
cat_features = ["family"]

X_xgb = train_df.drop("sales", axis=1)[["date"] + num_features + cat_features]
X_xgb['year'] = X_xgb['date'].dt.year
X_xgb['month'] = X_xgb['date'].dt.month
X_xgb['day'] = X_xgb['date'].dt.day
X_xgb = X_xgb.drop("date", axis=1)
X_xgb = pd.get_dummies(X_xgb)
y = train_df["sales"]



train_dmatrix = xgb.DMatrix(X_xgb, label=y)
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'eta': 0.1,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
}

In [16]:
model = xgb.train(params, train_dmatrix, num_boost_round=100)

In [17]:
xgb_test = test_df.drop("date", axis=1)
xgb_test = pd.get_dummies(xgb_test)
test_dmatrix = xgb.DMatrix(xgb_test)

In [18]:
xgb_pred = model.predict(test_dmatrix)

In [19]:
len(xgb_pred)

28512

In [20]:
xgb_pred

array([   1.9185929,    1.9185929,    2.4299207, ..., 1512.0491   ,
        366.1884   ,   46.5642   ], dtype=float32)